# 在 Google Colab 上运行 pyVideoTrans 语音转语音流程

本 Notebook 将安装 pyVideoTrans 所需依赖，并演示如何在 Colab 中通过 **index-tts2** 克隆原说话人音色。建议先阅读 `docs/google_colab_s2s.md` 获取完整部署指南。

## 1. 克隆仓库（每个会话运行一次）
如果你已经在会话中下载过项目，可以跳过此步骤。

In [ ]:
# 克隆仓库并切换到项目目录
import os, sys, pathlib
ROOT = pathlib.Path('/content/pyvideotrans')
if not ROOT.exists():
    !git clone https://github.com/jianchang512/pyvideotrans.git {ROOT}
os.chdir(ROOT)
sys.path.insert(0, str(ROOT))
print('Working directory:', os.getcwd())

## 2. 安装运行依赖
安装 Colab 精简依赖集，其中包含适配 CUDA 12.1 的 Faster-Whisper。

In [ ]:
# 安装 pyVideoTrans 在 Colab 运行所需的依赖
!pip install -U pip
!pip install -r requirements-colab.txt

## 3. （可选）挂载 Google Drive
如果希望直接从 Drive 读写文件，可取消注释并运行此单元。

In [ ]:
# 取消注释后挂载 Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

## 4. 准备输入媒体
可以通过 Colab 界面上传，也可以使用下面的示例代码从网络下载测试视频。

In [ ]:
# 下载示例视频，或替换为自己的文件
import pathlib, urllib.request, urllib.error\nsample_dir = pathlib.Path('/content/media')\nsample_dir.mkdir(parents=True, exist_ok=True)\nsample_path = sample_dir / 'sample.mp4'\nif not sample_path.exists():\n    url = 'https://github.com/jianchang512/pyvideotrans/releases/download/v0.0/sample.mp4'\n    try:\n        urllib.request.urlretrieve(url, sample_path)\n    except urllib.error.URLError:\n        print('Download failed, please upload your own clip instead.')\nsample_path

## 5. 启动语音到语音翻译
确保 `index-tts2` WebUI 可以被 Colab 访问（如通过 ngrok/cloudflared 暴露公网地址），然后设置目标语言及接口地址。

In [ ]:
# 调用封装好的函数执行完整流程
from colab_s2s import run_speech_to_speech
result = run_speech_to_speech(
    input_path=sample_path,
    target_language='en',
    index_tts_url='http://127.0.0.1:7860',  # replace with your public endpoint
    whisper_model='large-v3',
    translate_backend='google',
    recognition_backend='faster-whisper',
    separate_vocals=True,
    voice_role='clone'
)
result

生成的 `result` 字典会给出输出媒体和字幕的路径，可在 Colab 文件浏览器中下载或移动到 Drive。

## 附：使用 gradio_client 快速验证 Index-TTS2 接口
如果你部署了自建的 Index-TTS2 服务，可先运行下方示例确认连通性，再执行完整流程。

In [ ]:
# 可选：使用 gradio_client 直接验证 Index-TTS2 是否可用
# from gradio_client import Client, handle_file
# client = Client("https://<your-indextts2-host>")
# preview = client.predict(
#     emo_control_method="Same as the voice reference",
#     prompt=handle_file('/content/sample_ref.wav'),
#     text="你好，这是一个测试片段。",
#     emo_ref_path=handle_file('/content/sample_ref.wav'),
#     api_name="/gen_single"
# )
# print(preview)
